<a href="https://colab.research.google.com/github/czarodziejszyn/ssne/blob/main/projekt/recursive_model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [23]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import torch
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim

In [3]:
pickle_path = "/content/drive/MyDrive/data/train.pkl"

with open(pickle_path, 'rb') as f:
    data = pickle.load(f)

In [6]:
print(f"data length: {len(data)}")
print(f"element type: {type(data[0])}")
print(f"song fragment: {data[0][0][:10]}")
print(f"song class: {data[0][1]}")

data length: 2939
element type: <class 'tuple'>
song fragment: [ -1.  -1.  -1.  -1. 144. 144. 144.  64.  67.   0.]
song class: 0


In [8]:
X, y = zip(*data)

## Rozkład klas

In [9]:
class_counts = Counter(y)
print("class counts:")
for key, value in class_counts.items():
    print(f"{key}: {value}")

class counts:
0: 1630
1: 478
2: 154
3: 441
4: 236


## Statystyki utworów

In [10]:
lengths = [len(song) for song in X]
print(f"average length: {np.mean(lengths)}")
print(f"max length: {np.max(lengths)}")
print(f"min length: {np.min(lengths)}")

average length: 436.50493365090165
max length: 6308
min length: 4


## Preprocessing

In [17]:
X, y = zip(*data)
y = np.array(y)

flattened = np.concatenate(X)
scaler = StandardScaler()
scaler.fit(flattened.reshape(-1, 1))

X_scaled = [torch.tensor(scaler.transform(np.array(song).reshape(-1, 1)).flatten(), dtype=torch.float32) for song in X]
lengths = torch.tensor([len(song) for song in X_scaled])

X_padded = pad_sequence(X_scaled, batch_first=True)

y_tensor = torch.tensor(y, dtype=torch.long)

X_train, X_val, y_train, y_val, len_train, len_val = train_test_split(
    X_padded, y_tensor, lengths, test_size=0.2, stratify=y, random_state=42
)

## Dataloadery

In [21]:
train_dataset = TensorDataset(X_train, y_train, len_train)
val_dataset = TensorDataset(X_val, y_val, len_val)

BATCH_SIZE = 64
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

## Model

In [24]:
class RNNClassifier(nn.Module):
    def __init__(self, input_size=1, hidden_size=128, num_layers=1, num_classes=5, dropout=0.3):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )

        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x, lengths):
        x = x.unsqueeze(-1)
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_output, (hn, cn) = self.lstm(packed)
        last_hidden = hn[-1]
        out = self.fc(last_hidden)
        return out

## Trening

In [25]:
def evaluate_model(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_x, batch_y, batch_lengths in data_loader:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            batch_lengths = batch_lengths.to(device)

            outputs = model(batch_x, batch_lengths)
            loss = criterion(outputs, batch_y)

            total_loss += loss.item() * batch_x.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == batch_y).sum().item()
            total += batch_y.size(0)

    avg_loss = total_loss / total
    return avg_loss, correct, total


In [27]:
def train_model(model, train_loader, val_loader, num_epochs=10, lr=1e-3, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(1, num_epochs + 1):
        model.train()
        train_loss, train_correct, total = 0.0, 0, 0

        for batch_x, batch_y, batch_lengths in train_loader:
            batch_x, batch_y, batch_lengths = batch_x.to(device), batch_y.to(device), batch_lengths.to(device)

            optimizer.zero_grad()
            outputs = model(batch_x, batch_lengths)

            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * batch_x.size(0)
            _, predicted = torch.max(outputs, 1)
            train_correct += (predicted == batch_y).sum().item()
            total += batch_x.size(0)

        train_loss /= total
        train_acc = train_correct / total

        val_loss, val_correct, val_total = evaluate_model(model, val_loader, criterion, device)

        print(f"Epoch {epoch:02d} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_correct / val_total:.4f}")

In [29]:
model = RNNClassifier(
    input_size=1,
    hidden_size=256,
    num_layers=2,
    num_classes=5,
    dropout=0.3
)

train_model(model, train_loader, val_loader, num_epochs=10)

Epoch 01 | Train Loss: 1.3528 | Train Acc: 0.5432 | Val Loss: 1.2681 | Val Acc: 0.5544
Epoch 02 | Train Loss: 1.2501 | Train Acc: 0.5619 | Val Loss: 1.2298 | Val Acc: 0.5510
Epoch 03 | Train Loss: 1.2215 | Train Acc: 0.5751 | Val Loss: 1.2353 | Val Acc: 0.5697
Epoch 04 | Train Loss: 1.2307 | Train Acc: 0.5700 | Val Loss: 1.2666 | Val Acc: 0.5102
Epoch 05 | Train Loss: 1.2301 | Train Acc: 0.5525 | Val Loss: 1.2309 | Val Acc: 0.5544
Epoch 06 | Train Loss: 1.2364 | Train Acc: 0.5547 | Val Loss: 1.1994 | Val Acc: 0.5544
Epoch 07 | Train Loss: 1.1973 | Train Acc: 0.5547 | Val Loss: 1.2038 | Val Acc: 0.5544
Epoch 08 | Train Loss: 1.2316 | Train Acc: 0.5547 | Val Loss: 1.2526 | Val Acc: 0.5544
Epoch 09 | Train Loss: 1.2522 | Train Acc: 0.5547 | Val Loss: 1.2409 | Val Acc: 0.5544
Epoch 10 | Train Loss: 1.2220 | Train Acc: 0.5547 | Val Loss: 1.2018 | Val Acc: 0.5544


In [31]:
class BidirectionalRNNClassifier(nn.Module):
    def __init__(self, input_size=1, hidden_size=128, num_layers=1, num_classes=5, dropout=0.3):
        super(BidirectionalRNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True # Dwukierunkowe
        )

        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x, lengths):
        x = x.unsqueeze(-1)
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_output, (hn, cn) = self.lstm(packed)
        last_hidden = torch.cat((hn[-2,:,:], hn[-1,:,:]), dim=1)
        out = self.fc(last_hidden)
        return out

class_counts = Counter(y)
total_samples = sum(class_counts.values())
class_weights = torch.tensor([total_samples / class_counts[i] for i in range(len(class_counts))], dtype=torch.float32)
if torch.cuda.is_available():
    class_weights = class_weights.to('cuda')

def train_model_weighted(model, train_loader, val_loader, class_weights, num_epochs=10, lr=1e-3, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(1, num_epochs + 1):
        model.train()
        train_loss, train_correct, total = 0.0, 0, 0

        for batch_x, batch_y, batch_lengths in train_loader:
            batch_x, batch_y, batch_lengths = batch_x.to(device), batch_y.to(device), batch_lengths.to(device)

            optimizer.zero_grad()
            outputs = model(batch_x, batch_lengths)

            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * batch_x.size(0)
            _, predicted = torch.max(outputs, 1)
            train_correct += (predicted == batch_y).sum().item()
            total += batch_x.size(0)

        train_loss /= total
        train_acc = train_correct / total

        val_loss, val_correct, val_total = evaluate_model(model, val_loader, criterion, device)

        print(f"Epoch {epoch:02d} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_correct / val_total:.4f}")

model_improved = BidirectionalRNNClassifier(
    input_size=1,
    hidden_size=256,
    num_layers=2,
    num_classes=5,
    dropout=0.3
)

train_model_weighted(model_improved, train_loader, val_loader, class_weights, num_epochs=50, lr=5e-4)

Epoch 01 | Train Loss: 1.5915 | Train Acc: 0.3050 | Val Loss: 1.5619 | Val Acc: 0.2568
Epoch 02 | Train Loss: 1.5269 | Train Acc: 0.3228 | Val Loss: 1.5251 | Val Acc: 0.3673
Epoch 03 | Train Loss: 1.5110 | Train Acc: 0.3028 | Val Loss: 1.5167 | Val Acc: 0.2551
Epoch 04 | Train Loss: 1.4747 | Train Acc: 0.3071 | Val Loss: 1.5050 | Val Acc: 0.3146
Epoch 05 | Train Loss: 1.4659 | Train Acc: 0.3471 | Val Loss: 1.5000 | Val Acc: 0.3180
Epoch 06 | Train Loss: 1.4219 | Train Acc: 0.4024 | Val Loss: 1.4734 | Val Acc: 0.3963
Epoch 07 | Train Loss: 1.4198 | Train Acc: 0.4058 | Val Loss: 1.4810 | Val Acc: 0.4456
Epoch 08 | Train Loss: 1.4023 | Train Acc: 0.4079 | Val Loss: 1.4798 | Val Acc: 0.3231
Epoch 09 | Train Loss: 1.3984 | Train Acc: 0.4224 | Val Loss: 1.4614 | Val Acc: 0.3486
Epoch 10 | Train Loss: 1.3786 | Train Acc: 0.4032 | Val Loss: 1.4575 | Val Acc: 0.4048
Epoch 11 | Train Loss: 1.4233 | Train Acc: 0.4198 | Val Loss: 1.4892 | Val Acc: 0.2874
Epoch 12 | Train Loss: 1.3793 | Train Acc: 

In [32]:
model_path = "/content/drive/MyDrive/rnn_classifier.pth"
torch.save(model_improved.state_dict(), model_path)

In [33]:
model_path = "/content/drive/MyDrive/rnn_classifier.pth"
loaded_model = BidirectionalRNNClassifier(
    input_size=1,
    hidden_size=256,
    num_layers=2,
    num_classes=5,
    dropout=0.3
)
loaded_model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [ ]:
train_model_weighted(loaded_model, train_loader, val_loader, class_weights, num_epochs=20, lr=5e-4)

Epoch 01 | Train Loss: 0.8827 | Train Acc: 0.6146 | Val Loss: 1.3836 | Val Acc: 0.4966
Epoch 02 | Train Loss: 0.8157 | Train Acc: 0.6380 | Val Loss: 1.4244 | Val Acc: 0.5119
Epoch 03 | Train Loss: 0.7482 | Train Acc: 0.6678 | Val Loss: 1.3995 | Val Acc: 0.4932
Epoch 04 | Train Loss: 0.7391 | Train Acc: 0.6635 | Val Loss: 1.4094 | Val Acc: 0.5459
Epoch 05 | Train Loss: 0.6846 | Train Acc: 0.6874 | Val Loss: 1.4285 | Val Acc: 0.5187
Epoch 06 | Train Loss: 0.6608 | Train Acc: 0.6997 | Val Loss: 1.5349 | Val Acc: 0.4983
Epoch 07 | Train Loss: 0.6213 | Train Acc: 0.7167 | Val Loss: 1.4433 | Val Acc: 0.5663
Epoch 08 | Train Loss: 0.6169 | Train Acc: 0.7291 | Val Loss: 1.4767 | Val Acc: 0.5561
